# Answering Business Questions Using SQL

The Chinook database is provided as a SQLite database file called chinook.db about online music purchases.

In [51]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

#### Q1: Select the first three albums that will be added to the store, from a list of four:
1. Regal: Hip-Hop
2. Red Tone: Punk
3. Meteor and the Girls: Pop
4. Slim Jim Bites: Blues 

Write a query that returns each genre, with the number of tracks sold in the USA:
* in absolute numbers
* in percentages.

In [52]:
%%sql
WITH u AS 
        ( -- Named subquery to include only sales in the USA
        SELECT 
        billing_country country,
        invoice_id
        FROM invoice 
        WHERE country = "USA"
        )
SELECT 
        g.name genre,
        COUNT(t.track_id) no_of_tracks_sold, 
        ROUND((COUNT(t.track_id) * 100.0) / ( -- subquery to give total number of tracks to find percentage 
                                            SELECT COUNT(il.track_id)
                                            FROM invoice i
                                            INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
                                            WHERE I.billing_country = "USA"
                                            ), 2) percentage_sold
FROM u
LEFT JOIN invoice_line il ON il.invoice_id = u.invoice_id
LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1 ORDER BY 3 DESC 
LIMIT 10;

Done.


genre,no_of_tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the results, I would recommend: 
1. Red Tone: Punk
2. Slim Jim Bites: Blues
3. Meteor and the Girls: Pop
Together, these generes make up about %18 percent of the total sales but rock contibutes to 53% of sales so if we can we should find a rock artist.

#### Q2: analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.
Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. 
Add any extra attributes for that employee that you find are relevant to the analysis.

In [53]:
%%sql
SELECT 
    e.first_name || ' ' || e.last_name emp_name,
    ROUND(SUM(i.total), 2) total,
    e.hire_date date_hired,
    e.title title
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY employee_id ORDER BY 2 DESC;

Done.


emp_name,total,date_hired,title
Jane Peacock,1731.51,2017-04-01 00:00:00,Sales Support Agent
Margaret Park,1584.0,2017-05-03 00:00:00,Sales Support Agent
Steve Johnson,1393.92,2017-10-17 00:00:00,Sales Support Agent


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

#### Q3: You have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [62]:
%%sql
WITH 
    country_cust AS
      (
        SELECT 
          c.country country,
          c.customer_id customer_id,
          COUNT(i.invoice_id) no_of_order,
          SUM(i.total) total        
      FROM customer c
      INNER JOIN invoice i ON i.customer_id = c.customer_id
      GROUP BY 1, 2
    ),
        
    country_other AS
    (
    SELECT 
        COUNT(customer_id) no_of_cust, 
        SUM(total) total,
        SUM(no_of_order) no_of_orders,
        CASE 
            WHEN COUNT(customer_id) = 1 THEN 'Other'
            ELSE country
            END AS new_country
    FROM country_cust
    GROUP BY country
    )
SELECT 
    new_country, 
    SUM(no_of_cust) no_of_cust, 
    SUM(total) total,
    SUM(total) / SUM(no_of_cust) avg_per_cust,
    SUM(total) / no_of_orders avg_order_val
FROM (
    SELECT
        co.*,
        CASE
            WHEN co.new_country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_other co
   )
GROUP BY 1 ORDER BY sort ASC, 3 DESC;

Done.


new_country,no_of_cust,total,avg_per_cust,avg_order_val
USA,13,1040.49,80.03769230769231,7.942671755725191
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.67999999999995,85.53599999999999,7.011147540983606
France,5,389.07,77.814,7.7814
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.51999999999998,81.83999999999999,8.768571428571429
Portugal,2,185.13,92.565,6.3837931034482756
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,109.49399999999999



Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India